## Generating a data model for CLIWOC

In [ ]:
from __future__ import annotations

import mdf_reader

In [ ]:
schema = "imma1"
data_file_path = "../../tests/data/063-714_2010-07_subset.imma"
data_raw = mdf_reader.read(data_file_path, data_model=schema)

In [ ]:
data_raw.data["c99"].head()

In [ ]:
line = data_raw.data["c99"].iloc[3].values[0]
line

In [ ]:
len(line)

In [ ]:
model_path = "../../mdf_reader/data_models/library/imma1_d730"
data_file_path = "../../tests/data/133-730_1776-10_subset.imma"

In [ ]:
data = mdf_reader.read(data_file_path, data_model_path=model_path)

In [ ]:
data.data[["c99_sentinal"]].head()

In [ ]:
data.data[["c99_logbook"]].c99_logbook.describe(include="all")

In [ ]:
from __future__ import annotations

import pandas as pd

pd.options.display.max_columns = None
data.data[["c99_voyage"]].c99_voyage.describe(include="all")

In [ ]:
data.data[["c99_voyage"]].c99_voyage.ZeroMeridian.head()

e.g. the ship types on this deck will be given in a tons of different languages. There is no code table for this variable in the CLIWOC website.

Would this information will be enough (e.g. for things that Liz is doing)?

In [ ]:
data.data[["c99_voyage"]].c99_voyage.Ship_type.dropna().head()

In [ ]:
data.data[["c99_data"]].c99_data.describe(include="all")

What about the different scales for the wind force, given different languages?

In [ ]:
data.data[["c99_data"]].c99_data.wind_force.head()

For some of this data there are some code tables but according to the mdf reader we need a single code table.

We would have to make a nested .json file I think ... with the main key as the language. More ideas on how to do this are welcome.

I have done some of the easy key tables including the Meridian one. You can find those under  [ICOADS.C99.Variable.json](https://github.com/glamod/mdf_reader/tree/test_ll/mdf_reader/data_models/library/imma1_d730/code_tables)